In [1]:
import pandas as pd 
import os, re, json

In [2]:
with open("polidict.json", "r") as f:
    converter = json.load(f)

In [3]:
converter

{'Presidential Race': {'1': {'Donald Trump': -1, 'Joe Biden': 1},
  '2': {'Donald Trump': -1, 'Kamala Harris': 1}},
 'Immigration': {'1': {'Increased a lot': -1,
   'Increased somewhat': -1,
   'Kept the same': 0,
   'Decreased somewhat': 1,
   'Decreased a lot': 1},
  '2': {'Strongly favor': -1,
   'Somewhat favor': -1,
   'Neither favor nor oppose': 0,
   'Somewhat oppose': 1,
   'Strongly oppose': 1},
  '3': {'A lot easier': 1,
   'Somewhat easier': 1,
   'About the same': 0,
   'Somewhat harder': -1,
   'A lot harder': -1}},
 'Abortion': {'1': {'Legal in all cases': 1,
   'Legal in most cases': 1,
   'Illegal in most cases': -1,
   'Illegal in all cases': -1},
  '2': {'Extremely well': 1,
   'Very well': 1,
   'Somewhat well': 0,
   'Not too well': -1,
   'Not at all well': -1},
  '3': {'Extremely well': -1,
   'Very well': -1,
   'Somewhat well': 0,
   'Not too well': 1,
   'Not at all well': 1},
  '4': {'Extremely well': -1,
   'Very well': -1,
   'Somewhat well': 0,
   'Not too 

origin

In [ ]:
columns = ["sec", "id", "result", "llm", "n", "-1", "0", "1", "2", "3", "4", "5"]
records = pd.DataFrame(columns=columns)

for  file in os.listdir("./csv"):
    if 'convert' in file:
        df = pd.read_csv(os.path.join("./csv", file))
        df['section'] = df['section'].apply(lambda x: 'Abortion' if isinstance(x, str) and x.startswith('Abortion') else x)
        
        for section in converter.keys():
            for id in converter[section].keys():
                sub_df = df[(df['section'] == section) & (df['question_id'].astype(str) == str(id))]
                sub_col = sub_df["choice"]
                converted_choices = sub_col[sub_col.notna() & (sub_col != "")].map(converter[section][id])

                if len(converted_choices) > 0:
                    values = {'sec': section, 'id': id, 'result':round(sum(converted_choices)/len(converted_choices), 4), 'llm':sub_df['model'].values[0], "n":len(converted_choices)}
                else:
                    values = {'sec': section, 'id': id, 'result':-999, 'llm':sub_df['model'].values[0], "n":len(converted_choices)}
                for value in converter[section][id].values():
                    values[str(value)] = (converted_choices == value).sum()
                
                records = pd.concat([records, pd.DataFrame([values])], ignore_index=True)


In [5]:
records['question'] = records['sec'] + ' ' + records['id'].astype(str)
new_col = ["llm", "sec", "id", "question", "result", "n", "-1", "0", "1", "2", "3", "4", "5"]
records = records[new_col]

records.to_csv('stat_records_origin.csv', index=False)

combine by model and section 

In [6]:
def average_result(group):
    filtered_results = group['result'][group['result'] != -999]
    return filtered_results.mean() if not filtered_results.empty else -999

In [7]:
sec_order = list(converter.keys())

In [8]:
grouped = records.groupby(['llm', 'sec'])
new_df = grouped.apply(average_result).reset_index()
new_df.columns = ['llm', 'sec', 'ave_result']

/tmp/ipykernel_1463635/927369172.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  new_df = grouped.apply(average_result).reset_index()


In [9]:
new_df['sec'] = pd.Categorical(new_df['sec'], categories=sec_order, ordered=True)
sorted_new_df = new_df.sort_values(by=['llm', 'sec'])

In [10]:
def process_result(value):
    if value < -100:
        return int(value)  # Convert to int if no value
    else:
        return round(value, 4)  # Round to 4 decimal places

In [13]:
sorted_new_df['ave_result'] = sorted_new_df['ave_result'].apply(process_result)

sorted_new_df.to_csv('stat_records_origin_combined.csv', index=False)

combined

In [ ]:
columns = ["sec", "id", "result", "llm", "n", "-1", "0", "1", "2", "3", "4", "5"]
records = pd.DataFrame(columns=columns)

for  file in os.listdir("./augment/augment_csv2"):
        df = pd.read_csv(os.path.join("./augment/augment_csv2", file))
        df['section'] = df['section'].apply(lambda x: 'Abortion' if isinstance(x, str) and x.startswith('Abortion') else x)
        
        for section in converter.keys():
            for id in converter[section].keys():
                sub_df = df[(df['section'] == section) & (df['question_id'].astype(str) == str(id))]
                sub_col = sub_df["choice"]
                converted_choices = sub_col[sub_col.notna() & (sub_col != "")].map(converter[section][id])

                if len(converted_choices) > 0:
                    values = {'sec': section, 'id': id, 'result':round(sum(converted_choices)/len(converted_choices), 4), 'llm':sub_df['model'].values[0], "n":len(converted_choices)}
                else:
                    values = {'sec': section, 'id': id, 'result':-999, 'llm':sub_df['model'].values[0], "n":len(converted_choices)}
                for value in converter[section][id].values():
                    values[str(value)] = (converted_choices == value).sum()
                
                records = pd.concat([records, pd.DataFrame([values])], ignore_index=True)


/tmp/ipykernel_1463635/2937631612.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  records = pd.concat([records, pd.DataFrame([values])], ignore_index=True)


In [15]:
records['question'] = records['sec'] + ' ' + records['id'].astype(str)
new_col = ["llm", "sec", "id", "question", "result", "n", "-1", "0", "1", "2", "3", "4", "5"]
records = records[new_col]

records.to_csv('stat_records_jailbreak.csv', index=False)

jailbreak - combine by model and section

In [6]:
def average_result(group):
    filtered_results = group['result'][group['result'] != -999]
    return filtered_results.mean() if not filtered_results.empty else -999

def process_result(value):
    if value < -100:
        return int(value)  # Convert to int if no value
    else:
        return round(value, 4)  # Round to 4 decimal places

In [17]:
sec_order = list(converter.keys())

grouped = records.groupby(['llm', 'sec'])
new_df = grouped.apply(average_result).reset_index()
new_df.columns = ['llm', 'sec', 'ave_result']

new_df['sec'] = pd.Categorical(new_df['sec'], categories=sec_order, ordered=True)
sorted_new_df = new_df.sort_values(by=['llm', 'sec'])

sorted_new_df['ave_result'] = sorted_new_df['ave_result'].apply(process_result)

/tmp/ipykernel_1463635/4149121625.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  new_df = grouped.apply(average_result).reset_index()


In [18]:
sorted_new_df.to_csv('stat_records_jailbreak_combined.csv', index=False)

response rate, jailbreak only

In [ ]:
columns = ["sec", "id", "result", "llm", "n", "-1", "0", "1", "2", "3", "4", "5"]
records = pd.DataFrame(columns=columns)

for  file in os.listdir("./csv"):
        df = pd.read_csv(os.path.join("./csv", file))
        df['section'] = df['section'].apply(lambda x: 'Abortion' if isinstance(x, str) and x.startswith('Abortion') else x)
        
        for section in converter.keys():
            for id in converter[section].keys():
                sub_df = df[(df['section'] == section) & (df['question_id'].astype(str) == str(id))]
                sub_col = sub_df["choice"]
                converted_choices = sub_col[sub_col.notna() & (sub_col != "")].map(converter[section][id])

                if len(converted_choices) > 0:
                    values = {'sec': section, 'id': id, 'result':round(sum(converted_choices)/len(converted_choices), 4), 'llm':sub_df['model'].values[0], "n":len(converted_choices)}
                else:
                    values = {'sec': section, 'id': id, 'result':-999, 'llm':sub_df['model'].values[0], "n":len(converted_choices)}
                for value in converter[section][id].values():
                    values[str(value)] = (converted_choices == value).sum()
                
                records = pd.concat([records, pd.DataFrame([values])], ignore_index=True)

/tmp/ipykernel_2229763/3281269441.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  records = pd.concat([records, pd.DataFrame([values])], ignore_index=True)


In [5]:
records['question'] = records['sec'] + ' ' + records['id'].astype(str)
new_col = ["llm", "sec", "id", "question", "result", "n", "-1", "0", "1", "2", "3", "4", "5"]
records = records[new_col]

records.to_csv('stat_records_jailbreak_only.csv', index=False)

response rate, jailbreak only, combined by section

In [7]:
sec_order = list(converter.keys())

grouped = records.groupby(['llm', 'sec'])
new_df = grouped.apply(average_result).reset_index()
new_df.columns = ['llm', 'sec', 'ave_result']

new_df['sec'] = pd.Categorical(new_df['sec'], categories=sec_order, ordered=True)
sorted_new_df = new_df.sort_values(by=['llm', 'sec'])

sorted_new_df['ave_result'] = sorted_new_df['ave_result'].apply(process_result)

/tmp/ipykernel_2229763/4149121625.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  new_df = grouped.apply(average_result).reset_index()


In [8]:
sorted_new_df.to_csv('stat_records_jailbreak_only_combined.csv', index=False)